In [1]:
import pygsheets
import pandas as pd
import os
from dotenv import load_dotenv,find_dotenv

Connecting to service account

In [2]:
load_dotenv(find_dotenv())
gc = pygsheets.authorize(service_file=os.environ.get('JSON_FILE'))

Connecting to spreadsheet

In [3]:
sh = gc.open_by_url(os.environ.get('SPD_FILE_URL'))

Retrieving spreadsheet id

In [4]:
sh.id

'16wchX_JnBwscIfKZbANCsgSGnabCMRJ9r-72R1ayR68'

Retrieving worksheet title

In [9]:
sh.title

'template'

Retrieving all the worksheet data related to the spreadsheet

In [10]:
sh.worksheets()

[<Worksheet 'games_sales' index:0>,
 <Worksheet 'tryNew' index:1>,
 <Worksheet 'test' index:2>]

We can see that the data is in the form of a list of containing index and the name of each worksheet.

Connect to a single worksheet using it's index.

In [15]:
sht = sh.worksheet(property="index",value="0")
print(sht.title)

games_sales


Connect to a single worksheet using it's title.

In [16]:
sht = sh.worksheet(property="title",value="tryNew")
print(sht.title)

tryNew


Add a new worksheet to the spreadsheet.

In [17]:
sh.add_worksheet(title="New Sheet")

<Worksheet 'New Sheet' index:3>

In [18]:
print(sh.worksheets())

[<Worksheet 'games_sales' index:0>, <Worksheet 'tryNew' index:1>, <Worksheet 'test' index:2>, <Worksheet 'New Sheet' index:3>]


We can see that a new worksheet named "New Sheet" has been added to the spreadsheet.

Optional arguments in the add_worksheet function.

In [20]:
# if we want the data present in a existing worksheet to the new worksheet
sht = sh.worksheet(property="title",value="tryNew")
sh.add_worksheet(title="Copy Sheet", src_worksheet=sht)

<Worksheet 'Copy Sheet' index:5>

In [21]:
print(sh.worksheets())

[<Worksheet 'games_sales' index:0>, <Worksheet 'tryNew' index:1>, <Worksheet 'test' index:2>, <Worksheet 'New Sheet' index:3>, <Worksheet 'Copy Sheet' index:5>]


Delete a worksheet from the spreadsheet.

In [30]:
copy = sh.worksheet(property="title",value="Copy Sheet")
sh.del_worksheet(copy)

In [31]:
print(print(sh.worksheets()))

[<Worksheet 'games_sales' index:0>, <Worksheet 'tryNew' index:1>, <Worksheet 'test' index:2>, <Worksheet 'New Sheet' index:3>]
None


Share a worksheet with other users.

In [32]:
sh.share('jayakrishnan.c@ignitesol.com', role='writer', type='user', emailMessage='Here is the spreadsheet we talked about!')

Connecting the worksheet to the pandas.

In [34]:
wks = sh.worksheet(property="title",value="games_sales")
df = pd.DataFrame(wks.get_all_records())

In [35]:
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Kinect Adventures!,X360,2010,Misc,Microsoft Game Studios,14.97,4.94,0.24,1.67,21.82
1,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
2,Grand Theft Auto V,X360,2013,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38
3,Call of Duty: Modern Warfare 3,X360,2011,Shooter,Activision,9.03,4.28,0.13,1.32,14.76
4,Call of Duty: Black Ops,X360,2010,Shooter,Activision,9.67,3.73,0.11,1.13,14.64


Filtering Values present in the dataframe.

In [45]:
#retrieve the name of top three highesting grossing games which had eu_sales greater than na_sales
df2 = df.loc[df['EU_Sales'] > df['NA_Sales']].nlargest(3,columns=['Global_Sales'])

In [46]:
df2

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
1,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
5,Call of Duty: Black Ops 3,PS4,2015,Shooter,Activision,5.77,5.81,0.35,2.31,14.24
6,Call of Duty: Black Ops II,PS3,2012,Shooter,Activision,4.99,5.88,0.65,2.52,14.03


Let's move this dataframe to a new worksheet.

In [47]:
sh.worksheets()

[<Worksheet 'games_sales' index:0>,
 <Worksheet 'tryNew' index:1>,
 <Worksheet 'test' index:2>,
 <Worksheet 'New Sheet' index:3>]

In [48]:
wks = sh.worksheet(property="title",value="New Sheet")

In [49]:
wks.set_dataframe(df2,start='A1') #start is the cell where the dataframe will be inserted

In [50]:
wks.get_all_records()#let's check it the data has been inserted.

[{'Name': 'Grand Theft Auto V',
  'Platform': 'PS3',
  'Year': 2013,
  'Genre': 'Action',
  'Publisher': 'Take-Two Interactive',
  'NA_Sales': 7.01,
  'EU_Sales': 9.27,
  'JP_Sales': 0.97,
  'Other_Sales': 4.14,
  'Global_Sales': 21.4},
 {'Name': 'Call of Duty: Black Ops 3',
  'Platform': 'PS4',
  'Year': 2015,
  'Genre': 'Shooter',
  'Publisher': 'Activision',
  'NA_Sales': 5.77,
  'EU_Sales': 5.81,
  'JP_Sales': 0.35,
  'Other_Sales': 2.31,
  'Global_Sales': 14.24},
 {'Name': 'Call of Duty: Black Ops II',
  'Platform': 'PS3',
  'Year': 2012,
  'Genre': 'Shooter',
  'Publisher': 'Activision',
  'NA_Sales': 4.99,
  'EU_Sales': 5.88,
  'JP_Sales': 0.65,
  'Other_Sales': 2.52,
  'Global_Sales': 14.03}]